In [1]:
import numpy as np
import json_tricks


inputs = json_tricks.load('inputs.json')
answer = {}


# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a normal vector to this hyperspace $\mathbf w$

So that the hyperplane is given by equation:

$\left<\mathbf w, \mathbf x - \mathbf x_0\right> = 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [2]:
def distance_to_hyperplane_1(y, w, x0):
    # Compute the vector from x0 to y
    y_minus_x0 = y - x0
    
    # Compute the dot product of w and (y - x0)
    dot_product = np.dot(w, y_minus_x0)
    
    # Compute the norm of w
    norm_w = np.linalg.norm(w)
    
    # Compute the distance
    distance = np.abs(dot_product) / norm_w
    
    return distance

In [3]:
answer['task1'] = []
for one_input in inputs['task1']:
    answer['task1'].append(
        distance_to_hyperplane_1(**one_input))

# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 
    
    $A = \begin{bmatrix}
    | & | & & | \\
    \mathbf a_1 & \mathbf a_2 & \dots & \mathbf a_K \\
    | & | & & | \\
    \end{bmatrix}$

So that the hyperplane is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [4]:
def distance_to_hyperplane_2(y, A, x0):
    # Compute the QR decomposition of A
    Q, R = np.linalg.qr(A)
    
    # The normal vector w is the last column of Q when A is full rank
    # If A is not full rank, we need to find the orthogonal complement
    # Here, we assume A is full rank for simplicity
    w = Q[:, -1]
    
    # Compute the vector from x0 to y
    y_minus_x0 = y - x0
    
    # Compute the dot product of w and (y - x0)
    dot_product = np.dot(w, y_minus_x0)
    
    # Compute the norm of w
    norm_w = np.linalg.norm(w)
    
    # Compute the distance
    distance = np.abs(dot_product) / norm_w
    
    return distance

In [5]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(
        distance_to_hyperplane_2(**one_input))

# Task

You are given 2 linear hyperspaces, each defined by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 

So that each of the hyperplanes is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

Find distance between these hyperplanes.

In [6]:
def distance_to_hyperplane_3(A, x0, B, y0):
    res = 0
    ## YOUR CODE HERE
    return res

In [7]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(
        distance_to_hyperplane_3(**one_input))

In [8]:
json_tricks.dump(answer, '.answer.json')

'{"task1": [1.1611015532036637, 1.2169686348578783, 3.8171960993534175, 2.1577367083828145, 17.53449972356085, 13.749230523405476, 0.11441551070947108, 0.9750002110024923, 2.0481280123278074, 0.4178554470186725, 7.299732493572025, 5.405127802564358, 2.0541374149458744, 6.71079347281296, 3.03653640842388, 9.498185286556286, 3.7630890450319083, 1.8641906049463526, 10.670770020027398, 5.291772589866002, 0.912245460839306, 0.20732586478679277, 12.816310409006174, 1.6373225835853016, 6.291571550369559, 20.239709319388293, 16.466083091274903, 13.474719001142073, 5.2761044442380856, 0.7250110520819842, 4.519523979103389, 4.225904366704887, 4.775521858228876, 13.912436341952132, 7.24644943484495, 9.313787360469487, 1.0596535411727601, 5.682048493486052, 4.5807379490262905, 3.3300869947532497, 5.307227776030219, 5.6272294256742645, 4.913975701062781, 4.802205375719392, 6.501355432613329, 11.47165060247919, 1.1319709719592113, 4.803995448402746, 3.272807297357087, 15.777869833110813, 4.517699029